In [2]:
# import idx2numpy for extraction of image data (plus numpy and matplotlib)
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

from PIL import Image

2023-07-14 16:22:31.274962: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 16:22:31.621668: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 16:22:31.624248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 16:22:33.730590: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Read and preprocess data

In [3]:
#read mapping file as dataframe
mapping = pd.read_csv("data/emnist-balanced-mapping.txt", sep=" ", header=None)
mapping = mapping.rename(columns={0: "label", 1: "ascii"})

#add "translated" label as "char" variable
"""
label is in data,
ascii is the corresponding unicode code,
char is the 'translation'
"""
mapping.loc[:, "char"] = mapping.ascii.apply(chr)

mapping

,label,ascii,char
0,0,48,0
1,1,49,1
2,2,50,2
3,3,51,3
4,4,52,4
5,5,53,5
6,6,54,6
7,7,55,7
8,8,56,8
9,9,57,9


In [4]:
#use idx method convert_from_file() for extraction into array - separate for train and test
train_img = idx2numpy.convert_from_file("data/emnist-balanced-train-images-idx3-ubyte")
test_img = idx2numpy.convert_from_file("data/emnist-balanced-test-images-idx3-ubyte")

In [5]:
#same for the labels files
train_lab = idx2numpy.convert_from_file("data/emnist-balanced-train-labels-idx1-ubyte")
test_lab = idx2numpy.convert_from_file("data/emnist-balanced-test-labels-idx1-ubyte")

In [6]:
#normalize image data and reshape image data for the channels dimension
train_img = np.reshape(train_img, (-1, 28, 28, 1)) / 255
test_img = np.reshape(test_img, (-1, 28, 28, 1)) / 255

In [7]:
#transpose tilted image data
train_img = np.transpose(train_img, (0, 2, 1, 3))
test_img = np.transpose(test_img, (0, 2, 1, 3))

In [8]:
#encode the label data
train_lab_enc = to_categorical(train_lab)
test_lab_enc = to_categorical(test_lab)

# Model

In [12]:
#set up model via function
def initialize_model():

    model = models.Sequential()

    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(28,28,1), activation="relu", padding="same"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Conv2D(32, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(47, activation="relu"))
    
    model.add(layers.Dense(47, activation="softmax"))
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [13]:
mod = initialize_model()

In [14]:
es = EarlyStopping(patience=20, restore_best_weights=True)

mod.fit(train_img, train_lab_enc, validation_split=0.2, callbacks=[es], epochs=100, batch_size=32)

Epoch 1/100


2023-07-14 16:40:48.573840: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 282992640 exceeds 10% of free system memory.


2820/2820 [==============================] - 52s 18ms/step - loss: 0.9353 - accuracy: 0.7138 - val_loss: 0.5601 - val_accuracy: 0.8167
Epoch 2/100
2820/2820 [==============================] - 50s 18ms/step - loss: 0.4951 - accuracy: 0.8322 - val_loss: 0.4668 - val_accuracy: 0.8373
Epoch 3/100
2820/2820 [==============================] - 49s 17ms/step - loss: 0.4276 - accuracy: 0.8509 - val_loss: 0.4272 - val_accuracy: 0.8479
Epoch 4/100
2820/2820 [==============================] - 50s 18ms/step - loss: 0.3900 - accuracy: 0.8623 - val_loss: 0.3923 - val_accuracy: 0.8597
Epoch 5/100
2820/2820 [==============================] - 50s 18ms/step - loss: 0.3662 - accuracy: 0.8697 - val_loss: 0.3843 - val_accuracy: 0.8618
Epoch 6/100
2820/2820 [==============================] - 50s 18ms/step - loss: 0.3459 - accuracy: 0.8760 - val_loss: 0.3902 - val_accuracy: 0.8615
Epoch 7/100
2820/2820 [==============================] - 51s 18ms/step - loss: 0.3333 - accuracy: 0.8789 - val_loss: 0.3698 - val_

# Predicting generated input

In [15]:
def preproc(file):
    """
    load image from file, convert to grey scale and apply linear transformations + normalization
    """
    img = Image.open(file) 
    img = img.convert(mode="L") 
    img_arr = np.asarray(img) 
    img_arr = (img_arr * (-1) + 255) / 255 
    img_arr = img_arr.reshape((1,28,28,1))
    
    return img_arr

In [35]:
def output(image_array, model=mod):
    prediction = np.argmax(model.predict(image_array))
    mapping = pd.read_csv("data/emnist-balanced-mapping.txt", sep=" ", header=None) #TO DO Pfad anpassen
    mapping = mapping.rename(columns={0: "label", 1: "ascii"})
    
    return chr(mapping.loc[prediction, 'ascii'])

In [36]:
print(output(input_N, mod))

1/1 [==============================] - 0s 30ms/step
N
